# Build Feature Layers For Mapping

> **1.** Create dataframe from MySQL table containing allocated system

> **2.** Process child and ei data into a feature layer in ArcGIS which allows you to view a map of all the data

In [1]:
import pandas as pd
import numpy as np
from bson import ObjectId

import keyring
import mysql.connector

import arcgis
from arcgis.gis import GIS
import datetime as dt
import getpass
from IPython.display import HTML

from arcgis.geocoding import geocode
from arcgis.features import Feature, FeatureSet, FeatureLayerCollection, use_proximity, FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.network.analysis import generate_origin_destination_cost_matrix

In [2]:
username = "Glodanale"
password = keyring.get_password("arcgis", "Glodanale")
gis = GIS("https://www.arcgis.com", "Glodanale", password)

In [3]:
def combine_address(row):
    return f"{row['Address']}, {row['City']}, {row['State']} {row['Zipcode']}"

In [4]:
def pullData(table):
    connection = mysql.connector.connect(
        host = "localhost",
        port = 3306,
        user = 'root',
        database = "expandinghorizons",
        password = "dB79@dG2024!"
    )

    cursor = connection.cursor()
    
    query = ("Select c.child_id, c.Address, c.City, c.State, c.Zipcode, c.Longitude, c.Latitude, ld.ei_id_1 "
             "FROM child c "
             f"join {table} ld on c.child_id = ld.child_id_1 "
             "WHERE ld.child_id_2 is null and ld.ei_id_2 is null and ld.assigned = 1")
    cursor.execute(query)
    child_info = cursor.fetchall()
    child = pd.DataFrame(child_info, columns=["ID", "Address", "City", "State", "Zipcode", "Longitude", "Latitude", "Assigned_EI"])
    child["Role"] = "child"
    
    query = ("SELECT ei_id, Address, City, State, Zipcode, Longitude, Latitude, ei_id "
             "from ei")
    cursor.execute(query)
    ei_info = cursor.fetchall()
    ei = pd.DataFrame(ei_info, columns=["ID", "Address", "City", "State", "Zipcode", "Longitude", "Latitude", "Assigned_EI"])
    ei["Role"] = "ei"
    
    allData = pd.concat([ei, child], ignore_index=True)
    allData["Full_Address"] = allData.apply(combine_address, axis=1)
    featureSetDF = allData.drop(columns=["Address", "City", "State", "Zipcode"])
    
    cursor.close()
    connection.close()
    print(featureSetDF)
    
    return featureSetDF

In [5]:
def buildFeatureSet(df):
    features = []

    for index, row in df.iterrows():
        attributes = {"ID": row['ID'], "Address": row['Full_Address'], "Assigned_EI": row["Assigned_EI"], "Role": row['Role']}
        geometry = {"x": row["Longitude"], 'y': row["Latitude"]}
        feature = {"attributes": attributes, "geometry": geometry}
        features.append(feature)
        
    location_json = {
        "features": features,
        "spatialReference": {"wkid": 4326, "lastestWkid": 4326},
        "geometryType": "esriGeometryPoint",
        "fields": [
            {"name": "ID", "type": "esriFieldTypeString", "alias": "ID", "length": "50"},
            {"name": "Address", "type": "esriFieldTypeString", "alias": "Address", "length": "100"},
            {"name": "Assigned_EI", "type": "esriFieldTypeString", "alias": "Assigned_EI", "length": "50"},
            {"name": "Role", "type": "esriFieldTypeString", "alias": "Role", "length": "50"}
        ]
    }
        
    feature_set = FeatureSet.from_dict(location_json)
    return feature_set

In [6]:
def createFeatureLayer(feature_set, name):
    feature_layer_item = gis.content.import_data(feature_set, title=f"Feature Layer {name}")
    feature_layer_url = feature_layer_item.url
    #print("Feature Layer URL:", feature_layer_url)
    
    return feature_layer_item

In [7]:
def analyzeFeatureLayer(feature_layer_item):
    feature_layer = feature_layer_item.layers[0]
    features = feature_layer.query()

    for feature in features:
        print(feature)
        print("\n")

In [8]:
def process(table, name):
    df = pullData(table)
    feature_set = buildFeatureSet(df)
    feature_layer_item = createFeatureLayer(feature_set, name)
    analyzeFeatureLayer(feature_layer_item)
    
    print("Feature Layer Built")

In [9]:
process("locationdataalgo3", "Algorithm 3 Allocation")

                           ID  Longitude   Latitude               Assigned_EI  \
0    65d75a2ce09e67e025e8cd30 -82.372108  36.319485  65d75a2ce09e67e025e8cd30   
1    65d75bb0e09e67e025e8cd31 -81.977836  36.722124  65d75bb0e09e67e025e8cd31   
2    65d75c59e09e67e025e8cd32 -82.944595  36.197052  65d75c59e09e67e025e8cd32   
3    65d75c87e09e67e025e8cd33 -82.415306  36.305367  65d75c87e09e67e025e8cd33   
4    65d75cb9e09e67e025e8cd34 -82.234230  36.391361  65d75cb9e09e67e025e8cd34   
..                        ...        ...        ...                       ...   
204  65d8bcaae09e67e025e8cd87 -82.668076  36.578304  65d75dfae09e67e025e8cd3b   
205  65d8bd34e09e67e025e8cd88 -82.569412  36.574497  65d75dfae09e67e025e8cd3b   
206  65d8bdb5e09e67e025e8cd89 -82.660362  36.483902  65d75dfae09e67e025e8cd3b   
207  65e5d57838f826f292671ffd -82.374298  36.409046  65d75dd5e09e67e025e8cd3a   
208  65e5fe7438f826f292672025 -82.504990  36.419319  65d75dd5e09e67e025e8cd3a   

      Role                 

In [10]:
process("locationdataalgo4", "Algorithm 4 Allocation")

                           ID  Longitude   Latitude               Assigned_EI  \
0    65d75a2ce09e67e025e8cd30 -82.372108  36.319485  65d75a2ce09e67e025e8cd30   
1    65d75bb0e09e67e025e8cd31 -81.977836  36.722124  65d75bb0e09e67e025e8cd31   
2    65d75c59e09e67e025e8cd32 -82.944595  36.197052  65d75c59e09e67e025e8cd32   
3    65d75c87e09e67e025e8cd33 -82.415306  36.305367  65d75c87e09e67e025e8cd33   
4    65d75cb9e09e67e025e8cd34 -82.234230  36.391361  65d75cb9e09e67e025e8cd34   
..                        ...        ...        ...                       ...   
204  65d8bdb5e09e67e025e8cd89 -82.660362  36.483902  65d75dfae09e67e025e8cd3b   
205  65d8c2bde09e67e025e8cd92 -82.691795  36.523399  65d75dfae09e67e025e8cd3b   
206  65d8c331e09e67e025e8cd93 -82.995255  36.356022  65d75dfae09e67e025e8cd3b   
207  65e5d57838f826f292671ffd -82.374298  36.409046  65d75a2ce09e67e025e8cd30   
208  65e5fe7438f826f292672025 -82.504990  36.419319  65d75d08e09e67e025e8cd36   

      Role                 

In [11]:
process("locationdataalgo5", "Algorithm 5 Allocation")

                           ID  Longitude   Latitude               Assigned_EI  \
0    65d75a2ce09e67e025e8cd30 -82.372108  36.319485  65d75a2ce09e67e025e8cd30   
1    65d75bb0e09e67e025e8cd31 -81.977836  36.722124  65d75bb0e09e67e025e8cd31   
2    65d75c59e09e67e025e8cd32 -82.944595  36.197052  65d75c59e09e67e025e8cd32   
3    65d75c87e09e67e025e8cd33 -82.415306  36.305367  65d75c87e09e67e025e8cd33   
4    65d75cb9e09e67e025e8cd34 -82.234230  36.391361  65d75cb9e09e67e025e8cd34   
..                        ...        ...        ...                       ...   
204  65d8bb48e09e67e025e8cd85 -82.679001  36.544052  65d75dfae09e67e025e8cd3b   
205  65d8bd34e09e67e025e8cd88 -82.569412  36.574497  65d75dfae09e67e025e8cd3b   
206  65d76b1ee09e67e025e8cd4a -83.002129  36.412067  65d75dfae09e67e025e8cd3b   
207  65e5d57838f826f292671ffd -82.374298  36.409046  65d75dd5e09e67e025e8cd3a   
208  65e5fe7438f826f292672025 -82.504990  36.419319  65d75dd5e09e67e025e8cd3a   

      Role                 